In [1]:
import os
import multiprocessing

import tqdm
import pandas as pd
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
import torch.nn.functional as F


import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import degree

In [2]:
# device = torch.device("cpu")
device = torch.device("cuda")

In [3]:
# df_train = pd.read_csv(
#     os.path.join("data", "train.csv")
# )
# df_attrs = pd.read_csv(
#     os.path.join("data", "attr.csv")
# )

In [4]:
# node_feature_cols = [
#     "age",
#     "city_id",
#     "sex",
#     "school",
#     "university",
# ]

# edge_feature_cols = [
#     "t",
#     "x2",
#     "x3",
# ]

# adj_col_names = ["u", "v"]

# def prepare_graph_data(ego_id: int):
#     df_edges = df_train.loc[df_train.loc[:, "ego_id"] == ego_id]
#     df_features = df_attrs.loc[df_attrs.loc[:, "ego_id"] == ego_id]

#     unique_u = set(df_edges.loc[:, "u"])
#     unique_v = set(df_edges.loc[:, "v"])
#     unique_nodes_with_attrs = set(df_features.loc[:, "u"])

#     max_node_idx = max(max(unique_u), max(unique_v))
#     fake_nodes = [idx for idx in range(max_node_idx + 1) if idx not in unique_nodes_with_attrs]
#     if len(fake_nodes) > 0:
#         fake_features = [-1 for _ in fake_nodes]
#         fake_node_to_attrs = {
#             "ego_id": ego_id,
#             "u": list(fake_nodes),
#             "age": fake_features,
#             "city_id": fake_features,
#             "sex": fake_features,
#             "school": fake_features,
#             "university": fake_features,
#         }
#         fake_nodes_df = pd.DataFrame(fake_node_to_attrs)
#         fake_nodes_df = pd.concat([df_features, fake_nodes_df]).sort_values(by="u")

#         result = Data(
#             x=torch.tensor(fake_nodes_df.loc[:, node_feature_cols].to_numpy(), dtype=torch.float32),
#             edge_index=torch.tensor(df_edges.loc[:, adj_col_names].to_numpy(), dtype=torch.int64).T,
#             edge_attr=torch.tensor(df_edges.loc[:, edge_feature_cols].fillna(-1).to_numpy(), dtype=torch.float32),
#             y=torch.tensor(df_edges.loc[:, "x1"].to_numpy(), dtype=torch.float32),
#         ).to(device)

#         return result

In [5]:
# graph_dataset = []
# for ego_id in tqdm.tqdm(df_train.loc[:, "ego_id"].unique()):
#     ego_graph = prepare_graph_data(ego_id)
#     if ego_graph:
#         graph_dataset.append(prepare_graph_data(ego_id))

100%|██████████| 61786/61786 [2:15:56<00:00,  7.58it/s]  


In [3]:
graph_dataset = torch.load(os.path.join("data", "preprocessed", "train_tensor.pt"))

In [7]:
# torch.save(
#     graph_dataset,
#     os.path.join("data", "preprocessed", "train_tensor.pt"),
# )

In [4]:
loader = DataLoader(graph_dataset, batch_size=512)

In [14]:
train_loader = DataLoader(graph_dataset[:int(len(graph_dataset) * 0.2)], batch_size=512)
val_loader = DataLoader(graph_dataset[int(len(graph_dataset) * 0.9):int(len(graph_dataset) * 1.)], batch_size=512)

In [15]:
class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = pyg_nn.SAGEConv((-1, -1), hidden_channels)
        self.conv2 = pyg_nn.SAGEConv((-1, -1), hidden_channels)
        self.lin1 = torch.nn.Linear(2 * hidden_channels + 3, hidden_channels // 2)
        self.lin2 = torch.nn.Linear(hidden_channels // 2, 1)

    def forward(self, x, edge_index, edge_attr):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        x = F.dropout(x, training=self.training, p=0.05)

        idx_u, idx_v = edge_index
        x = torch.cat([x[idx_u], x[idx_v], edge_attr], dim=-1)
        x = self.lin1(x).relu()
        x = F.dropout(x, training=self.training, p=0.05)
        x = self.lin2(x)
        return x.view(-1)

hidden_state = 32
model = Model(hidden_channels=hidden_state).to(device)

In [16]:
local_val_minimum = 1.835

In [17]:
# optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)

initial_lr = 0.05
reset_interval = 5

optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=reset_interval, gamma=0.1)

for epoch in range(1, 1000):
    total_loss = 0
    model.train()
    optimizer.zero_grad()
    for train_batch in train_loader:
        pred = model(train_batch.x, train_batch.edge_index, train_batch.edge_attr)
        target = train_batch.y
        loss = F.mse_loss(pred, target)
        total_loss += loss
        loss.backward()
        optimizer.step()
    scheduler.step()
    loss = total_loss / len(loader)

    model.eval()
    val_metric = 0
    for val_batch in val_loader:
        with torch.no_grad():
            pred = model(val_batch.x, val_batch.edge_index, val_batch.edge_attr)
            target = val_batch.y

        val_metric += mean_squared_error(pred.cpu(), target.cpu())
    val_metric = val_metric / len(val_loader)

    if val_metric < local_val_minimum:
        model_name = f"h_{hidden_state}_val_metric_{val_metric}.pt"
        torch.save(
            model.state_dict(),
            os.path.join("models", model_name),
        )
        print(f"save model {model_name}")
        local_val_minimum = val_metric
    
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, val metric: {val_metric}')

Epoch: 001, Loss: 1707664501374976.0000, val metric: 5040491.708333333
Epoch: 002, Loss: 9755741.0000, val metric: 107541432.0
Epoch: 003, Loss: 22346632.0000, val metric: 120371900.66666667
Epoch: 004, Loss: 20588526.0000, val metric: 99439249.33333333
Epoch: 005, Loss: 16629775.0000, val metric: 79661988.0
Epoch: 006, Loss: 14554009.0000, val metric: 78029159.0
Epoch: 007, Loss: 14274633.0000, val metric: 76400729.0
Epoch: 008, Loss: 13921498.0000, val metric: 74727689.33333333
Epoch: 009, Loss: 13663069.0000, val metric: 73015123.33333333
Epoch: 010, Loss: 13332778.0000, val metric: 71289809.33333333
Epoch: 011, Loss: 13170366.0000, val metric: 71115311.66666667
Epoch: 012, Loss: 13092648.0000, val metric: 70935834.0
Epoch: 013, Loss: 13038540.0000, val metric: 70751096.33333333
Epoch: 014, Loss: 12982532.0000, val metric: 70561389.33333333
Epoch: 015, Loss: 12930468.0000, val metric: 70367484.66666667
Epoch: 016, Loss: 12955266.0000, val metric: 70347607.0
Epoch: 017, Loss: 1296870

In [13]:
pred

tensor([0.6808, 0.6808, 0.6808,  ..., 0.6808, 0.6808, 0.6808], device='cuda:0',
       grad_fn=<ViewBackward0>)